In [ ]:
from abc import ABC, abstractmethod

class Vehiculo(ABC):
    def __init__(self, marca, modelo):
        self.marca = marca
        self.modelo = modelo
        self.conduccion = None  # Strategy

    def set_conduccion(self, estrategia):
        self.conduccion = estrategia

    def conducir(self):
        if self.conduccion:
            self.conduccion.conducir(self)
        else:
            print("No hay estrategia de conducción asignada.")

    @abstractmethod
    def tipo(self):
        pass

    def __eq__(self, other):
        return isinstance(other, Vehiculo) and self.marca == other.marca and self.modelo == other.modelo

    def __add__(self, other):
        if isinstance(other, Vehiculo):
            flota = ControlFlota.obtener_instancia()
            flota.agregar_vehiculo(other)
            return self
        raise TypeError("Solo se pueden agregar objetos de tipo Vehiculo a la flota")

#SUBCLASES

class Automovil(Vehiculo):
    def tipo(self):
        return "Automóvil"

class Camion(Vehiculo):
    def tipo(self):
        return "Camión"

class Motocicleta(Vehiculo):
    def tipo(self):
        return "Motocicleta"

#STRATEGY PATTERN :CONDUCION DINAMICA

class EstrategiaConduccion(ABC):
    @abstractmethod
    def conducir(self, vehiculo):
        pass

class ConduccionNormal(EstrategiaConduccion):
    def conducir(self, vehiculo):
        print(f"{vehiculo.tipo()} {vehiculo.marca} {vehiculo.modelo} conduce de manera normal.")

class ConduccionDeportiva(EstrategiaConduccion):
    def conducir(self, vehiculo):
        print(f"{vehiculo.tipo()} {vehiculo.marca} {vehiculo.modelo} conduce de manera deportiva.")

class ConduccionEconomica(EstrategiaConduccion):
    def conducir(self, vehiculo):
        print(f"{vehiculo.tipo()} {vehiculo.marca} {vehiculo.modelo} conduce de manera económica.")

#DECORATOR PATTERN : ASISTENCIA

class VehiculoDecorator(Vehiculo):
    def __init__(self, vehiculo):
        self._vehiculo = vehiculo
    
    def __getattr__(self, nombre): ##redirecciona los atributos al vehiculo decorado
        return getattr(self._vehiculo, nombre)

    def tipo(self):
        return self._vehiculo.tipo()

    def conducir(self):
        self._vehiculo.conducir()

    def __eq__(self, other):
        return self._vehiculo == other

    def __add__(self, other):
        return self._vehiculo + other

class AsistenciaEstacionamiento(VehiculoDecorator):
    def conducir(self):
        self._vehiculo.conducir()
        print("→ Asistencia de estacionamiento activada.")

class AsistenciaCarril(VehiculoDecorator):
    def conducir(self):
        self._vehiculo.conducir()
        print("→ Asistencia de mantenimiento de carril activada.")

#SINGLETONE PATTERN : CONTROL DE FLOTA
class ControlFlota:
    _instancia = None

    def __init__(self):
        if ControlFlota._instancia is not None:
            raise Exception("Esta clase es un singleton.")
        self.vehiculos = []

    @staticmethod
    def obtener_instancia():
        if ControlFlota._instancia is None:
            ControlFlota._instancia = ControlFlota()
        return ControlFlota._instancia

    def agregar_vehiculo(self, vehiculo):
        if vehiculo not in self.vehiculos:
            self.vehiculos.append(vehiculo)

    def listar_vehiculos(self):
        print("\n🚘 Flota actual:")
        for v in self.vehiculos:
            print(f"• {v.tipo()} - {v.marca} {v.modelo}")
        if not self.vehiculos:
            print("→ No hay vehículos en la flota.")

#USO EJEMPLO

if __name__ == "__main__":
    auto = Automovil("Toyota", "Corolla")
    moto = Motocicleta("Yamaha", "FZ")
    camion = Camion("Volvo", "FH")

    # Estrategias
    auto.set_conduccion(ConduccionEconomica())
    moto.set_conduccion(ConduccionDeportiva())
    camion.set_conduccion(ConduccionNormal())

    # Decoradores
    auto = AsistenciaEstacionamiento(auto)
    camion = AsistenciaCarril(camion)

    # Conducir
    auto.conducir()
    moto.conducir()
    camion.conducir()

    # Comparar vehículos
    otro_auto = Automovil("Toyota", "Corolla")
    print("\n¿Son iguales?", auto == otro_auto)  # True

    # Agregar vehículos a la flota usando +
    auto + moto
    auto + camion

    # Mostrar flota
    flota = ControlFlota.obtener_instancia()
    flota.listar_vehiculos()

Automóvil Toyota Corolla conduce de manera económica.
→ Asistencia de estacionamiento activada.
Motocicleta Yamaha FZ conduce de manera deportiva.
Camión Volvo FH conduce de manera normal.
→ Asistencia de mantenimiento de carril activada.

¿Son iguales? True


AttributeError: 'AsistenciaCarril' object has no attribute 'marca'